<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/credit_card_transactions_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Call libraries

In [2]:
# 1.0
import pandas as pd
import os

In [3]:
# 1.1
pd.__version__  # 1.3.5

'1.3.5'

In [4]:
# 1.2 Display cell outputs from multiple commands
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Mount google drive

In [5]:
# 2.0 Mount to ccma@fsm.ac.in
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
# 2.1 Check files
!ls /gdrive/MyDrive/credit_card_transactions/

card_transaction_v1.csv  credit_card_edges.csv	   creditCard_users.csv
card_transaction_v1.zip  creditCard_merchants.csv  sample_2002.csv


In [7]:
# 2.2 Change current directory
os.chdir("/gdrive/MyDrive/credit_card_transactions/")
os.listdir()

['card_transaction_v1.csv',
 'card_transaction_v1.zip',
 '.ipynb_checkpoints',
 'creditCard_users.csv',
 'creditCard_merchants.csv',
 'credit_card_edges.csv',
 'sample_2002.csv']

column names:  
User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?   
Sample data:  
0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,,No
0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,,No



### Read data and explore

In [23]:
# 3.0 Read data. File size is > 2gb
#     Takes around 
#     Feel free to include more fields (city, time of transaction etc) and analyse:
%%time
df = pd.read_csv("card_transaction_v1.csv",
                 usecols = ["User", "Year",  "Merchant Name","Merchant City", "Amount", "Is Fraud?"])

CPU times: user 26.1 s, sys: 6.44 s, total: 32.5 s
Wall time: 35.6 s


In [24]:
# 3.1 Our data:
df.head()
df.tail()
df.shape   # (24386900, 6)

,User,Year,Amount,Merchant Name,Merchant City,Is Fraud?
0,0,2002,$134.09,3527213246127876953,La Verne,No
1,0,2002,$38.48,-727612092139916043,Monterey Park,No
2,0,2002,$120.34,-727612092139916043,Monterey Park,No
3,0,2002,$128.95,3414527459579106770,Monterey Park,No
4,0,2002,$104.71,5817218446178736267,La Verne,No


,User,Year,Amount,Merchant Name,Merchant City,Is Fraud?
24386895,1999,2020,$-54.00,-5162038175624867091,Merrimack,No
24386896,1999,2020,$54.00,-5162038175624867091,Merrimack,No
24386897,1999,2020,$59.15,2500998799892805156,Merrimack,No
24386898,1999,2020,$43.12,2500998799892805156,Merrimack,No
24386899,1999,2020,$45.13,4751695835751691036,Merrimack,No


(24386900, 6)

In [25]:
# 3.2 Attempt should be made to
#     reduce data size:

df.dtypes

User              int64
Year              int64
Amount           object
Merchant Name     int64
Merchant City    object
Is Fraud?        object
dtype: object

In [26]:
# 3.3 Is data balanced?

df['Is Fraud?'].value_counts()
print("\n")
df['Is Fraud?'].value_counts(normalize = True)  # 99.78% vs 0.12%

No     24357143
Yes       29757
Name: Is Fraud?, dtype: int64

No     0.99878
Yes    0.00122
Name: Is Fraud?, dtype: float64

In [27]:
# 3.4 Remove '$' sign from 'Amount':
#     Takes time 15 secs:
%%time
df['Amount']=df['Amount'].str.replace("$", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


CPU times: user 12.4 s, sys: 1.11 s, total: 13.5 s
Wall time: 13.7 s


In [28]:
# 3.5 Convert 'Amount' to numeric:
df['Amount'] = pd.to_numeric(df['Amount'])

In [29]:
# 3.6 Data shape
df.shape   # rows: 243,86,900; cols: 6)

(24386900, 6)

### Take a sample of data

In [30]:
# 4.0 As RAM is limited, we will work
#     with a sample for the year 2002:

dfsample = df.loc[df['Year'] == 2002, :]

In [31]:
# 4.1 Sample size: 
dfsample.shape   # (350732, 6)

(350732, 6)

In [32]:
# 4.2 How many incidents of frauds?
dfsample['Is Fraud?'].value_counts()   # Yes: 139, No: 350593

No     350593
Yes       139
Name: Is Fraud?, dtype: int64

In [33]:
# 4.3 Should you like to save this sample
#     for latter quick reading?

dfsample.to_csv("sample_2002.csv", index = False)

In [34]:
# 4.4 We work with a copy of data
#     If, we make mistakes, we come back here:

df_sample = dfsample.copy()

In [35]:
# 4.5 And our data:
df_sample.head()

,User,Year,Amount,Merchant Name,Merchant City,Is Fraud?
0,0,2002,134.09,3527213246127876953,La Verne,No
1,0,2002,38.48,-727612092139916043,Monterey Park,No
2,0,2002,120.34,-727612092139916043,Monterey Park,No
3,0,2002,128.95,3414527459579106770,Monterey Park,No
4,0,2002,104.71,5817218446178736267,La Verne,No


### Change user IDs
> 1.0 Know unique user names and transform them to short names  
> 2.0 Prepare a dictionary of user-ids and proposed Ids  
> 3.0 Make changes to our dataset using the dictionary  

In [36]:
# 5.0 To distinguish userids on network graph,
#     we will prefix userids with 'u' and also assign
#      them a short name so that they fit within nodes 
#       when displayed on graph:

#    First get unique user names
u_user = df_sample['User'].unique()

In [ ]:
# 5.1
u_user.sort()
u_user
print("\n")  
len(u_user)  # 426

In [38]:
# 5.2 Get alternate names by prefixing userids with 'u'

# 5.2.1 Transform sorted-user array to a dataframe; Get its index as a column; Rename new column as 'alt_name'
#       So, alternate names are sequenial: 0,1,2,3.. and final names will be: u1,u2,u3...
dx = pd.DataFrame(u_user, columns = ["u_user"]).reset_index().rename(columns = {'index' : 'alt_name' })

# 5.2.2 Transform the new column to string
dx['alt_name'] = dx['alt_name'].apply(str)

# 5.2.3 Create a new column 'a' with a constant value:
dx['a'] = "u"

# 5.2.4 Concatenate 'a' column and 'alt_name' column:
dx['alt_name'] = dx['a'].str.cat(dx['alt_name']) 

# 5.2.5 Drop 'a'
_=dx.pop('a')

# 5.2.6 Check:
dx.head()

,alt_name,u_user
0,u0,0
1,u1,2
2,u2,4
3,u3,5
4,u4,15


In [ ]:
# 5.3 Prepare a dictionary having
#     old names (key) and new names (value) 
#     for further transformation:

# https://stackoverflow.com/a/20250947/3282777
map_dict = dict(zip(dx['u_user'], dx['alt_name']))
map_dict

In [41]:
# 5.4 Use 'replace' method to replace values in column 'User':

df_sample['User'] = df_sample['User'].replace(map_dict, inplace= False)

In [42]:
# 5.5 Check:
df_sample.head()

,User,Year,Amount,Merchant Name,Merchant City,Is Fraud?
0,u0,2002,134.09,3527213246127876953,La Verne,No
1,u0,2002,38.48,-727612092139916043,Monterey Park,No
2,u0,2002,120.34,-727612092139916043,Monterey Park,No
3,u0,2002,128.95,3414527459579106770,Monterey Park,No
4,u0,2002,104.71,5817218446178736267,La Verne,No


### Change merchant IDs
Same steps for changing long merchantIds with short IDs. We prefix these Ids with 'm'

In [43]:
# 6.0 Unique merchant ids
mn = df_sample['Merchant Name'].unique()

In [44]:
# 6.1 Sort them in place
mn.sort()

In [45]:
# 6.2 Prepare a dataframe of existing and alternate names:
dx = pd.DataFrame(mn, columns = ["mt"]).reset_index().rename(columns = {'index' : 'alt_name' })
dx['alt_name'] = dx['alt_name'].apply(str)
dx['a'] = "m"
dx['alt_name'] = dx['a'].str.cat(dx['alt_name']) 
_=dx.pop('a')
dx.head()

,alt_name,mt
0,m0,-9216029123349204090
1,m1,-9215609737857753742
2,m2,-9214558223928539670
3,m3,-9211415240299816455
4,m4,-9210704852233124461


In [ ]:
# 6.3 Create a dictionary for transformation:
# https://stackoverflow.com/a/20250947/3282777
map_dict = dict(zip(dx['mt'], dx['alt_name']))
map_dict

In [47]:
# 6.4 Transform values in 'Merchant Name' using map_dict:
#     Takes time 4 minutes
%%time
df_sample['Merchant Name'] = df_sample['Merchant Name'].replace(map_dict, inplace= False)

CPU times: user 15.9 s, sys: 3min 41s, total: 3min 56s
Wall time: 3min 58s


In [49]:
# 6.5 And check
df_sample.head()

,User,Year,Amount,Merchant Name,Merchant City,Is Fraud?
0,u0,2002,134.09,m7466,La Verne,No
1,u0,2002,38.48,m4998,Monterey Park,No
2,u0,2002,120.34,m4998,Monterey Park,No
3,u0,2002,128.95,m7393,Monterey Park,No
4,u0,2002,104.71,m8834,La Verne,No


### User Nodes   


#### Fields

>This node must have two <i>must</i> fields: Id and Label. Besides these two, to distinguish Users from Merchants, we also have a 'Cat' field. The 'Cat' field has two values: *Member* for Users and *Institution* for Merchants.  

> Besides, these three common field names, other fields may also be there and may carry any names.


> If a user has committed fraud, even once, we mark him with propensity to commit frauds. A column 'suspect' is added to records this propensity.  

In [50]:
## 7.0 Group by user to get user node charteristics:
grpd_user = df_sample.groupby(['User'])
user_nodes = grpd_user['Amount'].mean().reset_index()
user_nodes = user_nodes.round(decimals = 2)
user_nodes.head()

,User,Amount
0,u0,84.78
1,u1,41.82
2,u10,44.98
3,u100,100.45
4,u101,40.78


In [51]:
# 7.1 Which of the users have committed fraud
users_suspect = df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'User' ].unique()
len(users_suspect)  # 25

25

In [52]:
# 7.2 Add a column 'suspect' with default value of 'No'
user_nodes['suspect'] = "No" 

# 7.3 To distinguish user nodes from merchant nodes on the graph
#     we add a 'Cat' column.

user_nodes['Cat'] = 'Member'

In [53]:
# 7.4 Even if a user committed fraud once, we set value
#     in 'suspect' as 'Yes' for every transaction:

for i in users_suspect:
  user_nodes.loc[user_nodes['User'] == i, 'suspect' ] = "Yes"  

In [54]:
# 7.5 So how many of them?

user_nodes['suspect'].value_counts()  # No: 401, 'Yes: 25
user_nodes.head()

No     401
Yes     25
Name: suspect, dtype: int64

,User,Amount,suspect,Cat
0,u0,84.78,No,Member
1,u1,41.82,No,Member
2,u10,44.98,No,Member
3,u100,100.45,No,Member
4,u101,40.78,No,Member


In [55]:
# 7.6 Create a 'Label' column
#     that simply records userids.
#     Or, rather a copy of 'User' feature
#     'Label' column is a MUST in a graph:

user_nodes['Label'] = user_nodes['User']

# 7.7 Also a graph should have an 'Id' column
#     We rename 'User' field as 'Id'

user_nodes = user_nodes.rename(columns = {'User': "Id"})

In [56]:
# 7.8 our user nodes data:
user_nodes.head()

,Id,Amount,suspect,Cat,Label
0,u0,84.78,No,Member,u0
1,u1,41.82,No,Member,u1
2,u10,44.98,No,Member,u10
3,u100,100.45,No,Member,u100
4,u101,40.78,No,Member,u101


In [57]:
# 7.9 We also create a column 'suspect_n' that records
#     1 for 'Yes' fraud and 0 for 'No' fraud
#     It is simply a numeric transformation of 'suspect' field:
#     We do this as data manipulation capabilities within gephi
#     are limited:

user_nodes['suspect_n'] = user_nodes['suspect']
user_nodes['suspect_n'] = user_nodes['suspect_n'].map({"Yes":1 , "No" : 0})

In [58]:
# 7.10 Save usernodes to gdrive with semicolon separator:

user_nodes.to_csv("creditCard_users.csv",
                  index = False,
                  sep = ";"
                  )

### Merchant nodes

In [59]:
## 8.0 Group by merchant and get merchant charteristics:
grpd_merchant = df_sample.groupby(['Merchant Name'])
merchant_nodes = grpd_merchant['Amount'].mean().reset_index()
merchant_nodes = merchant_nodes.round(2)
merchant_nodes.head()

,Merchant Name,Amount
0,m0,104.40
1,m1,18.14
2,m10,186.43
3,m100,18.96
4,m1000,5.00


In [ ]:
# 8.1 With which one of the merchants fraud comitted:
df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'Merchant Name' ].unique()
merchant_suspect = df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'Merchant Name' ].unique()
len(merchant_suspect)

In [61]:
# 8.2 We record this also in 'victim':
merchant_nodes['victim'] = "No" 

# 8.3 Merchant 'Cat' we designate as 'Institution'
merchant_nodes['Cat'] = 'Institution'

In [62]:
# 8.3 Record which all merchants have propensity to become victims:

for i in merchant_suspect:
  merchant_nodes.loc[merchant_nodes['Merchant Name'] == i, 'victim' ] = "Yes"
  

In [63]:
# 8.4
merchant_nodes['victim'].value_counts()  # 10777, 76

No     10777
Yes       76
Name: victim, dtype: int64

In [64]:
# 8.5 Label and Id columns of Merchant nodes:

merchant_nodes['Label'] = merchant_nodes['Merchant Name']
merchant_nodes = merchant_nodes.rename(columns = {'Merchant Name': "Id"})
merchant_nodes.head()

,Id,Amount,victim,Cat,Label
0,m0,104.40,No,Institution,m0
1,m1,18.14,No,Institution,m1
2,m10,186.43,No,Institution,m10
3,m100,18.96,No,Institution,m100
4,m1000,5.00,No,Institution,m1000


In [65]:
# 8.5 We have a numeric field victim_n recording the
#     same information as 'victim':

merchant_nodes['victim_n'] = merchant_nodes['victim']
merchant_nodes['victim_n'] = merchant_nodes['victim_n'].map({"Yes":1 , "No" : 0})

In [66]:
# 8.5.1
merchant_nodes.head()

,Id,Amount,victim,Cat,Label,victim_n
0,m0,104.40,No,Institution,m0,0
1,m1,18.14,No,Institution,m1,0
2,m10,186.43,No,Institution,m10,0
3,m100,18.96,No,Institution,m100,0
4,m1000,5.00,No,Institution,m1000,0


In [67]:
# 8.6 Save merchant_nodes information to a file:

merchant_nodes.to_csv("creditCard_merchants.csv",
                      index = False,
                      sep = ";")

### Edges

Edges must have three fields: *Source* , *Target* and *Type*. *Type* field records if an edge is directed or undirected. Besides these three, it may have other fields also.

In [68]:
# 9.0 Group by User and Merchant Name:
#     We also wish to record the number of interactions between
#     customer and merchants. These will be edge weights:

edges = df_sample.groupby([df_sample['User'], df_sample['Merchant Name']]).size()

In [69]:
# 9.1 The size column has a name '0'. We need to rename it:

edges = df_sample[['User', 'Merchant Name']].groupby(['User', 'Merchant Name']).size().reset_index()
edges = edges.rename(columns = { 0 : "weight"})
edges.head()

,User,Merchant Name,weight
0,u0,m10039,3
1,u0,m10101,5
2,u0,m10183,1
3,u0,m10800,2
4,u0,m1118,5


In [70]:
# 9.2 Rename two other columns appropriately:

edges = edges.rename(columns = {'User': 'Source', 'Merchant Name': 'Target'})

In [71]:
# 9.3 Our edges are Undirected:

edges['Type']= 'Undirected'
edges.head()

,Source,Target,weight,Type
0,u0,m10039,3,Undirected
1,u0,m10101,5,Undirected
2,u0,m10183,1,Undirected
3,u0,m10800,2,Undirected
4,u0,m1118,5,Undirected


In [72]:
# 9.4 Finally save edges information to a file:

edges.to_csv("credit_card_edges.csv",
             index = False,
             sep = ";"
             )

In [ ]:
########### Done ###########